In [1]:
commonly_used_4_digit = {
    '0000', '1111', '2222', '3333', '4444', '5555', '6666', '7777', '8888', '9999', 
    '1234', '2345', '3456', '4567', '5678', '6789', '7890', '8901', '9012', 
    '4321', '3210', '9876', '8765', '7654', '6543', '5432', '1098', '0987', 
    '0123', '1000', '2000', '1122', '2211',
    '1212', '2121', '1313', '3131', '0101', '1010', '0202', '2020',
    '1357', '2468', '0246', '1100', '0011', '1230', 
    '0001', '0010', '0100', 
    '0022', '2200',
}

In [2]:
import datetime

In [3]:
def format_date_part(part):
    return str(part).zfill(2)

In [4]:
def generate_demographic_patterns(date_string):
    patterns = set()
    if not date_string:
        return patterns
    try:
        date = datetime.datetime.strptime(date_string, '%Y-%m-%d')
    except ValueError:
        return patterns

    day = format_date_part(date.day)
    month = format_date_part(date.month)
    year_short = format_date_part(date.year % 100) # Last two digits of the year

    # Common 4-digit patterns
    patterns.add(f"{month}{day}") # MMDD
    patterns.add(f"{day}{month}") # DDMM
    patterns.add(f"{year_short}{month}") # YYMM
    patterns.add(f"{month}{year_short}") # MMYY
    patterns.add(f"{year_short}{day}") # YYDD
    patterns.add(f"{day}{year_short}") # DDYY

    return patterns

In [5]:
def check_mpin_strength_part_b(mpin, dob_self='', dob_spouse='', anniversary=''):
    if not isinstance(mpin, str) or len(mpin) != 4 or not mpin.isdigit():
        print(f"Error: '{mpin}' is not a valid 4-digit MPIN. Please enter exactly 4 digits.")
        return "INVALID"

    # Part A: Check if commonly used
    if mpin in commonly_used_4_digit:
        return "WEAK"

    # Part B: Check against demographics
    # Self DOB
    self_dob_patterns = generate_demographic_patterns(dob_self)
    if mpin in self_dob_patterns:
        return "WEAK"

    # Spouse DOB
    spouse_dob_patterns = generate_demographic_patterns(dob_spouse)
    if mpin in spouse_dob_patterns:
        return "WEAK"

    # Wedding Anniversary
    anniversary_patterns = generate_demographic_patterns(anniversary)
    if mpin in anniversary_patterns:
        return "WEAK"

    # If none of the above conditions are met, the MPIN is considered strong
    return "STRONG"

In [6]:
test_cases_part_ab = [
    # Part A: Commonly Used (no demographics)
    {'mpin': '1111', 'dob_self': '', 'dob_spouse': '', 'anniversary': '', 'expected_output': 'WEAK'},
    {'mpin': '1234', 'dob_self': '', 'dob_spouse': '', 'anniversary': '', 'expected_output': 'WEAK'},
    {'mpin': '0000', 'dob_self': '', 'dob_spouse': '', 'anniversary': '', 'expected_output': 'WEAK'},
    {'mpin': '9999', 'dob_self': '', 'dob_spouse': '', 'anniversary': '', 'expected_output': 'WEAK'},
    {'mpin': '0123', 'dob_self': '', 'dob_spouse': '', 'anniversary': '', 'expected_output': 'WEAK'},
    {'mpin': '4321', 'dob_self': '', 'dob_spouse': '', 'anniversary': '', 'expected_output': 'WEAK'},
    {'mpin': '1122', 'dob_self': '', 'dob_spouse': '', 'anniversary': '', 'expected_output': 'WEAK'},
    {'mpin': '5678', 'dob_self': '', 'dob_spouse': '', 'anniversary': '', 'expected_output': 'WEAK'}, 
    {'mpin': '8765', 'dob_self': '', 'dob_spouse': '', 'anniversary': '', 'expected_output': 'WEAK'},
       # Self DOB
    {'mpin': '0102', 'dob_self': '1990-02-01', 'dob_spouse': '', 'anniversary': '', 'expected_output': 'WEAK'}, # DDMM
    {'mpin': '0201', 'dob_self': '1990-01-02', 'dob_spouse': '', 'anniversary': '', 'expected_output': 'WEAK'}, # MMDD
    {'mpin': '9001', 'dob_self': '1990-01-02', 'dob_spouse': '', 'anniversary': '', 'expected_output': 'WEAK'}, # YYMM
    {'mpin': '0190', 'dob_self': '1990-01-02', 'dob_spouse': '', 'anniversary': '', 'expected_output': 'WEAK'}, # MMYY
    {'mpin': '0290', 'dob_self': '1990-02-01', 'dob_spouse': '', 'anniversary': '', 'expected_output': 'WEAK'}, # DDYY
    {'mpin': '9002', 'dob_self': '1990-02-01', 'dob_spouse': '', 'anniversary': '', 'expected_output': 'WEAK'}, # YYDD

    # Spouse DOB
    {'mpin': '0304', 'dob_self': '', 'dob_spouse': '1985-04-03', 'anniversary': '', 'expected_output': 'WEAK'},
    {'mpin': '0403', 'dob_self': '', 'dob_spouse': '1985-03-04', 'anniversary': '', 'expected_output': 'WEAK'},

    # Wedding Anniversary
    {'mpin': '0506', 'dob_self': '', 'dob_spouse': '', 'anniversary': '2000-06-05', 'expected_output': 'WEAK'},
    {'mpin': '0605', 'dob_self': '', 'dob_spouse': '', 'anniversary': '2000-05-06', 'expected_output': 'WEAK'},

    # Strong MPINs (not commonly used, no demographic match)
    {'mpin': '7891', 'dob_self': '1982-12-31', 'dob_spouse': '1983-01-01', 'anniversary': '2001-07-07', 'expected_output': 'STRONG'},
    {'mpin': '0007', 'dob_self': '1970-04-05', 'dob_spouse': '1972-06-10', 'anniversary': '1998-09-25', 'expected_output': 'STRONG'},
    {'mpin': '9871', 'dob_self': '1995-03-15', 'dob_spouse': '1988-07-20', 'anniversary': '2005-11-10', 'expected_output': 'STRONG'},

    # Invalid inputs
    {'mpin': '123', 'dob_self': '', 'dob_spouse': '', 'anniversary': '', 'expected_output': 'INVALID'}, # Invalid length
    {'mpin': 'ABCD', 'dob_self': '', 'dob_spouse': '', 'anniversary': '', 'expected_output': 'INVALID'}, # Non-digit characters
    {'mpin': '12345', 'dob_self': '', 'dob_spouse': '', 'anniversary': '', 'expected_output': 'INVALID'}, # Invalid length
    {'mpin': '', 'dob_self': '', 'dob_spouse': '', 'anniversary': '', 'expected_output': 'INVALID'}, # Empty string
    {'mpin': '1234', 'dob_self': 'invalid-date', 'dob_spouse': '', 'anniversary': '', 'expected_output': 'WEAK'}, # Common, invalid date is ignored
    {'mpin': '5678', 'dob_self': 'invalid-date', 'dob_spouse': '', 'anniversary': '', 'expected_output': 'WEAK'}, # Now common, invalid date is ignored
]

In [7]:
def run_tests_part_ab():
    print("--- Running Test Cases for Part A & B ---")
    all_passed = True
    for i, test in enumerate(test_cases_part_ab):
        mpin = test['mpin']
        dob_self = test['dob_self']
        dob_spouse = test['dob_spouse']
        anniversary = test['anniversary']
        expected = test['expected_output']
        actual = check_mpin_strength_part_b(mpin, dob_self, dob_spouse, anniversary)
        passed = (actual == expected)

        status = "PASSED" if passed else "FAILED"
        print(f"Test {i+1}: MPIN='{mpin}', DOB_Self='{dob_self}', DOB_Spouse='{dob_spouse}', Anniv='{anniversary}'")
        print(f"  Expected Strength: '{expected}'")
        print(f"  Actual Strength:   '{actual}'")
        print(f"  Status: {status}\n")

In [8]:
user_mpin = input("Please enter a 4-digit MPIN to check: ")
user_dob_self = input("Enter your Date of Birth (YYYY-MM-DD): ")
user_dob_spouse = input("Enter your Spouse's Date of Birth (YYYY-MM-DD): ")
user_anniversary = input("Enter your Wedding Anniversary (YYYY-MM-DD): ")

user_result = check_mpin_strength_part_b(user_mpin, user_dob_self, user_dob_spouse, user_anniversary)
print(f"\nYour MPIN '{user_mpin}' is: {user_result}")

Please enter a 4-digit MPIN to check:  1708
Enter your Date of Birth (YYYY-MM-DD):  2004-08-17
Enter your Spouse's Date of Birth (YYYY-MM-DD):  2003-06-14
Enter your Wedding Anniversary (YYYY-MM-DD):  2006-03-12



Your MPIN '1708' is: WEAK


In [9]:
run_tests_part_ab()

--- Running Test Cases for Part A & B ---
Test 1: MPIN='1111', DOB_Self='', DOB_Spouse='', Anniv=''
  Expected Strength: 'WEAK'
  Actual Strength:   'WEAK'
  Status: PASSED

Test 2: MPIN='1234', DOB_Self='', DOB_Spouse='', Anniv=''
  Expected Strength: 'WEAK'
  Actual Strength:   'WEAK'
  Status: PASSED

Test 3: MPIN='0000', DOB_Self='', DOB_Spouse='', Anniv=''
  Expected Strength: 'WEAK'
  Actual Strength:   'WEAK'
  Status: PASSED

Test 4: MPIN='9999', DOB_Self='', DOB_Spouse='', Anniv=''
  Expected Strength: 'WEAK'
  Actual Strength:   'WEAK'
  Status: PASSED

Test 5: MPIN='0123', DOB_Self='', DOB_Spouse='', Anniv=''
  Expected Strength: 'WEAK'
  Actual Strength:   'WEAK'
  Status: PASSED

Test 6: MPIN='4321', DOB_Self='', DOB_Spouse='', Anniv=''
  Expected Strength: 'WEAK'
  Actual Strength:   'WEAK'
  Status: PASSED

Test 7: MPIN='1122', DOB_Self='', DOB_Spouse='', Anniv=''
  Expected Strength: 'WEAK'
  Actual Strength:   'WEAK'
  Status: PASSED

Test 8: MPIN='5678', DOB_Self='', 